<a href="https://colab.research.google.com/github/NeuroCodeInnovations/Estimacion_Peso_Cuyes/blob/main/Yolov8/Segmentacion2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd/content/drive/MyDrive/PDSEI-Segmentacion

/content/drive/MyDrive/PDSEI-Segmentacion


In [ ]:
import os
HOME = os.getcwd()

In [ ]:
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

/content/drive/MyDrive/PDSEI-Segmentacion
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-5otk6dbx
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-5otk6dbx
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=16a9437fe1d92e9032f0ad484578832ffb130003266d8f8c40aef63d621b659f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5lbnuu_s/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment_anything


In [ ]:
%cd {HOME}
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

/content/drive/MyDrive/PDSEI-Segmentacion
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 907.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00


In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/drive/MyDrive/PDSEI-Segmentacion/weights/sam_vit_h_4b8939.pth ; exist: True


In [ ]:
import torch

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [ ]:
mask_predictor = SamPredictor(sam)

In [ ]:
# Obtencion de Factores y pixeles del cuy en una misma linea de codigo
import os
import cv2
import numpy as np
from scipy.stats import mode
import supervision as sv
import csv

# Definición de los grupos
def generar_grupos():
    grupos = []
    grupos.extend([f"B_CUY_01_{i:03d}" for i in range(1, 119)])
    grupos.extend([f"CUY_01_{i:03d}" for i in range(1, 119)])
    grupos.extend([f"CUY_00_{i:03d}" for i in range(2, 227)])
    return grupos

# Leer los pesos del archivo
def leer_pesos(path_pesos):
    pesos_por_grupo = {}
    with open(path_pesos, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Saltar la cabecera
        for row in reader:
            filename, weight = row
            grupo = "_".join(filename.split("_")[:3])
            pesos_por_grupo[grupo] = float(weight)
    return pesos_por_grupo

# Generar la lista de pesos en el orden de los grupos
def generar_lista_pesos(grupos, pesos_por_grupo):
    lista_pesos = []
    for grupo in grupos:
        grupo_clave = grupo[2:] if grupo.startswith("B_") else grupo
        peso = pesos_por_grupo.get(grupo_clave, 0)
        lista_pesos.append(peso)
    return lista_pesos

# Configuración de rutas
path = '/content/drive/MyDrive/PDSEI/data'
path_annotations = '/content/drive/MyDrive/PDSEI/ImagenesAnotadas'
path_pesos = '/content/drive/MyDrive/PDSEI/data/Pesos.txt'
images_folder = os.path.join(path, "images")

# Generar la lista de grupos y pesos
grupos = generar_grupos()
pesos_por_grupo = leer_pesos(path_pesos)
lista_pesos = generar_lista_pesos(grupos, pesos_por_grupo)

# Crear listas para almacenar resultados
pixeles_prom = []
factores_cuy_promedio = []  # Lista de factores promedio por cuy

# Procesar cada grupo
for grupo, peso in zip(grupos, lista_pesos):
    print(f"\nProcesando grupo: {grupo} (Peso: {peso}g)")

    # Filtrar imágenes del grupo
    imagenes_grupo = [
        img for img in os.listdir(images_folder)
        if img.startswith(grupo) and img.endswith(".jpg")
    ]

    pixeles_por_imagen = []
    factores_individuales = []  # Para almacenar factores de cada imagen de este cuy

    for IMAGE_NAME in imagenes_grupo:
        IMAGE_PATH = os.path.join(images_folder, IMAGE_NAME)
        TXT_PATH = os.path.join(path_annotations, IMAGE_NAME.replace('.jpg', '.txt'))

        # Leer la imagen
        imagen = cv2.imread(IMAGE_PATH)
        if imagen is None:
            print(f"No se pudo leer la imagen: {IMAGE_PATH}")
            continue

        altura, ancho = imagen.shape[:2]
        pixeles_totales = altura * ancho

        # Leer el archivo .txt y realizar predicción
        try:
            with open(TXT_PATH, 'r') as file:
                line = file.readline().strip()
                valores = line.split()
                if len(valores) >= 5:
                    xc, yc, an, al = map(float, valores[1:5])
                    x1 = (xc - an / 2) * ancho
                    y1 = (yc - al / 2) * altura
                    x2 = (xc + an / 2) * ancho
                    y2 = (yc + al / 2) * altura

                    box = np.array([x1, y1, x2, y2])

                    image_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

                    mask_predictor.set_image(image_rgb)

                    masks, scores, logits = mask_predictor.predict(
                        box=box,
                        multimask_output=True
                    )

                    box_annotator = sv.BoxAnnotator(
                        color=sv.Color.RED,
                        color_lookup=sv.ColorLookup.INDEX
                    )
                    mask_annotator = sv.MaskAnnotator(
                        color=sv.Color.RED,
                        color_lookup=sv.ColorLookup.INDEX
                    )

                    detections = sv.Detections(
                        xyxy=sv.mask_to_xyxy(masks=masks),
                        mask=masks
                    )
                    detections = detections[detections.area == np.max(detections.area)]

                    if detections.area[0] > pixeles_totales * 0.1:
                        print(f"Imagen {IMAGE_NAME} excluida por detección demasiado grande.")
                        continue

                    numero_pixeles_mascara = detections.area[0]
                    pixeles_por_imagen.append(numero_pixeles_mascara)

                    factor_cuy = peso / numero_pixeles_mascara if numero_pixeles_mascara > 0 else 0
                    factores_individuales.append(factor_cuy)

                    source_image = box_annotator.annotate(scene=imagen.copy(), detections=detections)
                    segmented_image = mask_annotator.annotate(scene=imagen.copy(), detections=detections)

                    font = cv2.FONT_HERSHEY_SIMPLEX
                    text = f"Pixeles: {numero_pixeles_mascara}, Factor: {factor_cuy:.2f} g/px"
                    cv2.putText(segmented_image, text, (10, 30), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                    sv.plot_images_grid(
                        images=[source_image, segmented_image],
                        grid_size=(1, 2),
                        titles=['Original', 'Mascara']
                    )
        except FileNotFoundError:
            print(f"Archivo TXT no encontrado para la imagen: {TXT_PATH}")
            continue

    if factores_individuales:
        promedio_factor_cuy = sum(factores_individuales) / len(factores_individuales)
        factores_cuy_promedio.append(promedio_factor_cuy)
        print(f"Factor promedio para {grupo}: {promedio_factor_cuy:.2f}")
    else:
        factores_cuy_promedio.append(0)
        print(f"No se calcularon factores para el grupo {grupo}.")

# Imprimir resultados finales
print("\nFactores promedio por cuy:")
print(factores_cuy_promedio)

In [ ]:
# Mismo código que el anterior, solo que consume menos recursos al no mostrar las imagenes

import os
import cv2
import numpy as np
from scipy.stats import mode
import supervision as sv
import csv

# Definición de los grupos
def generar_grupos():
    grupos = []
    grupos.extend([f"B_CUY_01_{i:03d}" for i in range(1, 119)])
    grupos.extend([f"CUY_01_{i:03d}" for i in range(1, 119)])
    grupos.extend([f"CUY_00_{i:03d}" for i in range(2, 227)])
    return grupos

# Leer los pesos del archivo
def leer_pesos(path_pesos):
    pesos_por_grupo = {}
    with open(path_pesos, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Saltar la cabecera
        for row in reader:
            filename, weight = row
            grupo = "_".join(filename.split("_")[:3])
            pesos_por_grupo[grupo] = float(weight)
    return pesos_por_grupo

# Generar la lista de pesos en el orden de los grupos
def generar_lista_pesos(grupos, pesos_por_grupo):
    lista_pesos = []
    for grupo in grupos:
        grupo_clave = grupo[2:] if grupo.startswith("B_") else grupo
        peso = pesos_por_grupo.get(grupo_clave, 0)
        lista_pesos.append(peso)
    return lista_pesos

# Configuración de rutas
path = '/content/drive/MyDrive/PDSEI/data'
path_annotations = '/content/drive/MyDrive/PDSEI/ImagenesAnotadas'
path_pesos = '/content/drive/MyDrive/PDSEI/data/Pesos.txt'
images_folder = os.path.join(path, "images")

# Generar la lista de grupos y pesos
grupos = generar_grupos()
pesos_por_grupo = leer_pesos(path_pesos)
lista_pesos = generar_lista_pesos(grupos, pesos_por_grupo)

# Crear listas para almacenar resultados
pixeles_prom = []
factores_cuy_promedio = []  # Lista de factores promedio por cuy

# Procesar cada grupo
for grupo, peso in zip(grupos, lista_pesos):
    print(f"\nProcesando grupo: {grupo} (Peso: {peso}g)")

    # Filtrar imágenes del grupo
    imagenes_grupo = [
        img for img in os.listdir(images_folder)
        if img.startswith(grupo) and img.endswith(".jpg")
    ]

    pixeles_por_imagen = []
    factores_individuales = []  # Para almacenar factores de cada imagen de este cuy

    for IMAGE_NAME in imagenes_grupo:
        IMAGE_PATH = os.path.join(images_folder, IMAGE_NAME)
        TXT_PATH = os.path.join(path_annotations, IMAGE_NAME.replace('.jpg', '.txt'))

        # Leer la imagen
        imagen = cv2.imread(IMAGE_PATH)
        if imagen is None:
            print(f"No se pudo leer la imagen: {IMAGE_PATH}")
            continue

        altura, ancho = imagen.shape[:2]
        pixeles_totales = altura * ancho

        # Leer el archivo .txt y realizar predicción
        try:
            with open(TXT_PATH, 'r') as file:
                line = file.readline().strip()
                valores = line.split()
                if len(valores) >= 5:
                    xc, yc, an, al = map(float, valores[1:5])
                    x1 = (xc - an / 2) * ancho
                    y1 = (yc - al / 2) * altura
                    x2 = (xc + an / 2) * ancho
                    y2 = (yc + al / 2) * altura

                    box = np.array([x1, y1, x2, y2])

                    image_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

                    mask_predictor.set_image(image_rgb)

                    masks, scores, logits = mask_predictor.predict(
                        box=box,
                        multimask_output=True
                    )

                    detections = sv.Detections(
                        xyxy=sv.mask_to_xyxy(masks=masks),
                        mask=masks
                    )
                    detections = detections[detections.area == np.max(detections.area)]

                    if detections.area[0] > pixeles_totales * 0.1:
                        print(f"Imagen {IMAGE_NAME} excluida por detección demasiado grande.")
                        continue

                    numero_pixeles_mascara = detections.area[0]
                    pixeles_por_imagen.append(numero_pixeles_mascara)

                    factor_cuy = peso / numero_pixeles_mascara if numero_pixeles_mascara > 0 else 0
                    factores_individuales.append(factor_cuy)

        except FileNotFoundError:
            print(f"Archivo TXT no encontrado para la imagen: {TXT_PATH}")
            continue

    if factores_individuales:
        promedio_factor_cuy = sum(factores_individuales) / len(factores_individuales)
        factores_cuy_promedio.append(promedio_factor_cuy)
        print(f"Factor promedio para {grupo}: {promedio_factor_cuy:.2f}")
    else:
        factores_cuy_promedio.append(0)
        print(f"No se calcularon factores para el grupo {grupo}.")

# Imprimir resultados finales
print("\nFactores promedio por cuy:")
print(factores_cuy_promedio)



Procesando grupo: B_CUY_01_001 (Peso: 308.44256g)
Archivo TXT no encontrado para la imagen: /content/drive/MyDrive/PDSEI/ImagenesAnotadas/B_CUY_01_001_14 (1).txt
Archivo TXT no encontrado para la imagen: /content/drive/MyDrive/PDSEI/ImagenesAnotadas/B_CUY_01_001_12 (1).txt
Archivo TXT no encontrado para la imagen: /content/drive/MyDrive/PDSEI/ImagenesAnotadas/B_CUY_01_001_13 (1).txt
Archivo TXT no encontrado para la imagen: /content/drive/MyDrive/PDSEI/ImagenesAnotadas/B_CUY_01_001_11 (1).txt
Archivo TXT no encontrado para la imagen: /content/drive/MyDrive/PDSEI/ImagenesAnotadas/B_CUY_01_001_09 (1).txt
Archivo TXT no encontrado para la imagen: /content/drive/MyDrive/PDSEI/ImagenesAnotadas/B_CUY_01_001_10 (1).txt
Archivo TXT no encontrado para la imagen: /content/drive/MyDrive/PDSEI/ImagenesAnotadas/B_CUY_01_001_15 (1).txt
Archivo TXT no encontrado para la imagen: /content/drive/MyDrive/PDSEI/ImagenesAnotadas/B_CUY_01_001_08 (1).txt
Archivo TXT no encontrado para la imagen: /content/dr

In [ ]:
factores_cuy_promedio = [0.2224565905803666, 0.2781510790058912, 0.3152436262610019, 0.29757902060808683, 0.24922368296381586, 0.25561165439269407, 0.2924045919104414, 0.2484748349747883, 0.20281944181961165, 0.2858223369461197, 0.23866858030857996, 0.2258788938790636, 0.19817095169660298, 0, 0.2875216451029813, 0.3000456447510994, 0, 0.4027383351623902, 0.2753825178212201, 0, 0.2746428300557201, 0.28494360662086954, 0.28698214285130924, 0, 0.3480864397121311, 0, 0, 0.3684030887080882, 0, 0.324491254846821, 0.30474737200355956, 0.30274551148508966, 0.2778629709341964, 0.2896227570643375, 0, 0.3180831950467724, 0.28992295907142307, 0.19325684323779072, 0.17932815092670382, 0.24526448971981893, 0, 0.17653461706668735, 0, 0.43415862894635837, 0, 0.2618722594977791, 0.2980918183736168, 0.26215150209184646, 0, 0.304493190702996, 0.3248150524090265, 0.29894304111101677, 0.32379945853086345, 0.29486039200397507, 0.34559828892666267, 0.37938527167551006, 0.3082144555028913, 0.31997156649478625, 0.3032407498550265, 0.21720543000623985, 0.19113909071181368, 0.23559582954166539, 0.22996185883963577, 0.19260106526911258, 0, 0, 0.2415594313967847, 0.18456310276566945, 0.265910097270867, 0.21898722873651125, 0, 0.2233653861985937, 0.2617098234086342, 0, 0.20931455394277756, 0, 0.34059607097601013, 0.3893399105910967, 0.38110802835477076, 0.3598614812148735, 0.16968486236346092, 0, 0, 0.22684139161001848, 0.1884918523422418, 0, 0.16252456633088663, 0.1591147298183645, 0.20618786214061416, 0.3657916994149242, 0.3947496303880987, 0, 0.3824832238857637, 0.3752962469020976, 0.42061337348249556, 0.31470780479058796, 0.23339098606542333, 0.20797574301769742, 0.22199165163563414, 0.21350066048328017, 0, 0, 0.2762362432448226, 0.22083891008067885, 0.24227106853846422, 0, 0.30165351985654704, 0.28761119934277224, 0.37754682710890625, 0.3829598583135599, 0.3359302131147356, 0.32048175654208966, 0, 0.30451042662944966, 0.39091756426990204, 0.3056778288039846, 0.36714584769552927, 0.26293758775415477, 0.21301944690505673, 0.26120153899374854, 0.3220612874151475, 0.3050782991939921, 0.2782653751486921, 0.29630832980815325, 0.2935370305947837, 0.25682109334257636, 0.21830914729512893, 0.2796676534410427, 0.25463521878629936, 0.23698092598668113, 0.2282204907721506, 0, 0.33926970441510323, 0.32991361103263356, 0, 0.3991421353471756, 0.31937255118805485, 0, 0.3281555149717394, 0.314486382706686, 0.2823765533616836, 0, 0.3552735974358198, 0, 0, 0.34935194874592274, 0, 0.366704922746193, 0.3299178682882885, 0.30852351332137784, 0.30629655893914054, 0.3034115085516845, 0, 0.30837764202780554, 0.26518747841110984, 0.20552386766536315, 0.1881091621308217, 0.24467975833953784, 0, 0.1673962439262892, 0, 0.4180555014948666, 0, 0.2682748764318535, 0.2978721997157431, 0.2812632433759797, 0, 0.3072407601753281, 0.35284255154871264, 0.3217679660244376, 0.3184504745893896, 0.33220409706215553, 0.37273619892128035, 0.42160201766170174, 0.3304461895468916, 0.3140585629518983, 0.2980954523636443, 0.3000985023987739, 0.224957745081902, 0.29927347748722366, 0.24877003685196916, 0.18809247260409848, 0, 0, 0.2711076487002895, 0.20830319968372518, 0.2690174910597094, 0.25288590399891825, 0, 0.2625286778523648, 0.23563081305002406, 0, 0.2514331991464402, 0, 0.43741710767175307, 0.4119379159712741, 0.3922798418388402, 0.3951744950442982, 0.2090555137722163, 0, 0, 0.20954886662833352, 0.16391007764280557, 0.1721845382876934, 0.1712452164862738, 0.21026045360218398, 0.19998873676013185, 0.40758793647366276, 0.43180119726169036, 0.4333536242342671, 0, 0.39192304078657175, 0.4395764142855698, 0.30315369883289695, 0.267841140203633, 0.21585695979643404, 0.20456646270731937, 0.23874124197778776, 0, 0, 0.2883105559913876, 0.2692819789081203, 0.2798686677050001, 0, 0.320854742761937, 0.31828796786109176, 0.3738858473999785, 0.40563392394891407, 0.34969034618459605, 0.30286382534638706, 0, 0.3573530234116782, 0.3985765755146088, 0.32217308522297866, 0.3633507201452521, 0.2649198492463218, 0.35113993788239334, 0.48706527261766264, 0.3262796458673504, 0.3202087520011818, 0.27382938672299356, 0, 0.315665733019318, 0.2896767902464266, 0.2330379526668426, 0.24682892837552753, 0, 0.2801651598381404, 0.316274250878754, 0.44469011907744205, 0.328095169951095, 0.36966963182858803, 0.491988420037631, 0.47411758554252775, 0.41276781810231744, 0.3465088278846431, 0.4520856373010047, 0, 0.4338121713360916, 0, 0.4803639346862792, 0.553909095839397, 0.3014815362893785, 0, 0, 0.5577632056910791, 0, 0.6119350694529815, 0.6201130433940771, 0.44319013554656034, 0, 0, 0, 0.4908172983841041, 0.6741656199232048, 0.7025547351563018, 0.6140901001099116, 0.8710962422453927, 0.7134957090490355, 0.815424373906027, 0.8625876691969548, 0.7289414963474619, 0, 0.8628111209370766, 0, 0, 0.6981270514069889, 0.6818442954134636, 0.7987762602555709, 0, 0.633043224258456, 0.7611055689584492, 0, 0.7490727911088262, 0.8193886521600863, 0.7419640681815884, 0.5905204830192794, 0.7265738688068527, 0.5608826852733043, 0.555541574722744, 0.6374584446628777, 0.5453541734150614, 0.6105377360447413, 0.7509809301541286, 0.7499776100161462, 0, 0.7699598382008975, 0.7224463438521724, 0, 0.8341405509173128, 0, 0, 0.7869660114984005, 0.6178111250132885, 0.7485537892056845, 0, 0, 0.6884149737544281, 0.6959123308412541, 0.6260208870369534, 0, 0, 0.568552267259233, 0.6022555020477555, 0, 0, 0.5563069210658373, 0.6815409159159159, 0.6049482266331884, 0.6166018897390051, 0.6241853326892975, 0.4542894477409137, 0.6580121529237106, 0.5568363698689024, 0.5742867094605604, 0.5170345511251451, 0.4744622845101513, 0.402491352392708, 0.4667854521953009, 0.5691971579221095, 0.48063295527772415, 0.5493151203533506, 0.5933490650143826, 0.5320586464660626, 0, 0.5348810948174041, 0.5106276716569826, 0.5170212453562255, 0.49443550249466667, 0.5636096806362157, 0, 0.618470992529489, 0.7143587875050801, 0.595372428942783, 0.654040506765098, 0, 0.4949896324210581, 0.553172003196763, 0.7845331290229862, 0, 0.4028985507246377, 0.7867507780868438, 0, 0, 0.7271271849992473, 0, 0.5633928571428571, 0.4969733376402889, 0.6740424466254735, 0.33840807215267077, 0.5444034118048128, 0.392216213757094, 0.36193645070522223, 0, 0.48852947246289835, 0.32343718981635644, 0.4022887323943662, 0.33159409305755677, 0.36631465998447676, 0, 0.31268205753888406, 0.35211199012432726, 0.3232833230224798, 0.36842067875960727, 0.46480257978919703, 0, 0.5041862198977338, 0.5237603932503225, 0.4488182580711004, 0, 0.47992290045208286, 0.5586662497466799, 0.5172782776754647, 0, 0.5381558028616852, 0.40016570008285, 0.713641637433796, 0.42909280500521374, 0.7390381895332391, 0.6826473686060017, 0.6050299251815823, 0, 0, 0.6523382882212159, 0, 0, 0.6209828486973259, 0.6109771201355844, 0.8980030000707304, 0.6400853580997986, 0.7496279631291065, 0, 0.6120218579234973, 0.5397688907414483, 0, 0.5994070843293147, 0.6989357954959808, 0.40095076119285666, 0.3959706772492768, 0, 0, 0.43963992757368026, 0, 0.6257144607199734, 0, 0.6868016341828783, 0.6499234487780134, 0.4670093382466293, 0.404979217406643, 0.38564086989010726, 0.6845086224586068, 0.4752176065441731, 0.5243638390464553, 0.627729588150545, 0.6049612717245375, 0.7986124988343125, 0, 0.7766688876146042, 0.6141963957337256, 0.6786036873704904, 0.5985915492957746, 0.6328618144209317, 0, 0.6191716216135554, 0, 0.6060144942961142, 0, 0, 0.5848374977811869, 0.5743922772721527, 0.6755674232309746, 0, 0.6164430216349513, 0.6010816359551961, 0.621598115771904, 0.5896150128891092, 0.652469337223231, 0.6430851063829788, 0.5895455704655143, 0.5649304893969742, 0.5422129801946235]

In [ ]:
from scipy.stats import mode
from collections import Counter

factores_invertidos = [1 / x for x in factores_cuy_promedio if x > 0]
conteo = Counter(factores_invertidos)
moda_invertida = max(conteo, key=conteo.get)
moda_original = 1 / moda_invertida

print(moda_invertida)
print(moda_original)

4.495259040836245
0.22245659058036657


In [ ]:
from scipy.stats import mode
moda = 0.22245659058036657

if factores_cuy_promedio:

    umbral = moda * 1.2
    # Filtrar los valores dentro del umbral
    factores_filtrados = [x for x in factores_cuy_promedio if x <= umbral]
    # Excluir los valores igual a 0 para el cálculo del promedio
    factores_filtrados_sin_ceros = [x for x in factores_filtrados if x > 0]

    promedio_final = (
        sum(factores_filtrados_sin_ceros) / len(factores_filtrados_sin_ceros)
        if factores_filtrados_sin_ceros
        else 0
    )
else:
    moda = 0
    umbral = 0
    factores_filtrados = []
    promedio_final = 0

# Imprimir resultados
print("Moda estadística:", moda)
print("Umbral:", umbral)
print("Lista filtrada:", factores_filtrados)
print("Lista filtrada sin ceros:", factores_filtrados_sin_ceros)
print("Promedio final:", promedio_final)


Moda estadística: 0.22245659058036657
Umbral: 0.26694790869643986
Lista filtrada: [0.2224565905803666, 0.24922368296381586, 0.25561165439269407, 0.2484748349747883, 0.20281944181961165, 0.23866858030857996, 0.2258788938790636, 0.19817095169660298, 0, 0, 0, 0, 0, 0, 0, 0, 0.19325684323779072, 0.17932815092670382, 0.24526448971981893, 0, 0.17653461706668735, 0, 0, 0.2618722594977791, 0.26215150209184646, 0, 0.21720543000623985, 0.19113909071181368, 0.23559582954166539, 0.22996185883963577, 0.19260106526911258, 0, 0, 0.2415594313967847, 0.18456310276566945, 0.265910097270867, 0.21898722873651125, 0, 0.2233653861985937, 0.2617098234086342, 0, 0.20931455394277756, 0, 0.16968486236346092, 0, 0, 0.22684139161001848, 0.1884918523422418, 0, 0.16252456633088663, 0.1591147298183645, 0.20618786214061416, 0, 0.23339098606542333, 0.20797574301769742, 0.22199165163563414, 0.21350066048328017, 0, 0, 0.22083891008067885, 0.24227106853846422, 0, 0, 0.26293758775415477, 0.21301944690505673, 0.26120153899

In [ ]:
factor_final = 0.22083622891943683

# Calculo del RMSE

In [ ]:
import os
import numpy as np
import cv2

def calcular_rmse_y_generar_archivo(factor_final, imagenes_dir, txt_dir, pesos_path, output_path):
    resultados = []

    # Leer el archivo de pesos
    try:
        with open(pesos_path, 'r') as f:
            pesos_data = {}
            for line in f:
                try:
                    nombre, peso = line.strip().split(',')
                    pesos_data[nombre] = float(peso)
                except ValueError:
                    print(f"Línea inválida en archivo de pesos: {line.strip()}")
    except Exception as e:
        print(f"Error al leer el archivo de pesos: {e}")
        return

    for archivo in os.listdir(imagenes_dir):
        if archivo.endswith('.jpg'):
            imagen_path = os.path.join(imagenes_dir, archivo)
            nombre_cuy = os.path.splitext(archivo)[0]
            txt_path = os.path.join(txt_dir, f"{nombre_cuy}.txt")

            if not os.path.exists(txt_path):
                continue

            # Leer la imagen
            imagen = cv2.imread(imagen_path)
            if imagen is None:
                print(f"No se pudo cargar la imagen: {imagen_path}")
                continue

            alto_img, ancho_img, _ = imagen.shape

            # Leer el peso real del cuy
            peso_cuy = pesos_data.get(f"{nombre_cuy}.jpg")
            if peso_cuy is None:
                print(f"No se encontró el peso para el cuy: {nombre_cuy}")
                continue

            # Leer las coordenadas del archivo de texto
            try:
                with open(txt_path, "r") as f:
                    linea = f.readline().strip().split()
                    _, x_centro_norm, y_centro_norm, ancho_norm, alto_norm = map(float, linea)

                    # Convertir coordenadas normalizadas a píxeles
                    ancho = int(ancho_norm * ancho_img)
                    alto = int(alto_norm * alto_img)

                    # Calcular el número de píxeles del cuadro delimitador
                    num_pixeles = ancho * alto

                    if num_pixeles == 0 or num_pixeles > 10000:
                        continue

                    # Calcular el peso estimado
                    peso_estimado = num_pixeles * factor_final

                    # Calcular el error cuadrático
                    error_cuadratico = (peso_estimado - peso_cuy) ** 2

                    resultados.append({
                        "nombre": nombre_cuy,
                        "peso_real": peso_cuy,
                        "peso_estimado": peso_estimado,
                        "error_cuadratico": error_cuadratico
                    })

            except Exception as e:
                print(f"Error al leer el archivo de coordenadas {txt_path}: {e}")
                continue

    # Calcular el RMSE
    errores_cuadraticos = [r["error_cuadratico"] for r in resultados]
    rmse = np.sqrt(np.mean(errores_cuadraticos)) if errores_cuadraticos else 0

    # Guardar resultados en un archivo
    with open(output_path, 'w') as f:
        f.write("Nombre de la imagen,Peso real (g),Peso estimado (g),RMSE (g)\n")
        for r in resultados:
            f.write(f"{r['nombre']},{r['peso_real']},{r['peso_estimado']},{np.sqrt(r['error_cuadratico'])}\n")

    # Ordenar resultados por RMSE
    resultados.sort(key=lambda x: np.sqrt(x["error_cuadratico"]))

    # Obtener los análisis solicitados
    top_5_bajos = resultados[:5]
    top_5_altos = resultados[-5:]
    mayores_a_200 = sum(1 for r in resultados if np.sqrt(r["error_cuadratico"]) > 200)
    menores_a_50 = sum(1 for r in resultados if np.sqrt(r["error_cuadratico"]) < 50)

    total = len(resultados)
    porcentaje_mayores_a_200 = (mayores_a_200 / total) * 100 if total > 0 else 0
    porcentaje_menores_a_50 = (menores_a_50 / total) * 100 if total > 0 else 0

    print(f"RMSE total: {rmse:.2f} g")
    print("Top 5 RMSE más bajos:")
    for r in top_5_bajos:
        print(f"{r['nombre']}: {np.sqrt(r['error_cuadratico']):.2f} g")

    print("\nTop 5 RMSE más altos:")
    for r in top_5_altos:
        print(f"{r['nombre']}: {np.sqrt(r['error_cuadratico']):.2f} g")

    print(f"\nPorcentaje de RMSE > 200 g: {porcentaje_mayores_a_200:.2f}%")
    print(f"Porcentaje de RMSE < 50 g: {porcentaje_menores_a_50:.2f}%")

# Configurar las rutas
directorio_imagenes = '/content/drive/MyDrive/PDSEI/Imagenes_Test_Anotadas'
directorio_txt = '/content/drive/MyDrive/PDSEI/Imagenes_Test_Anotadas'
pesos_path = '/content/drive/MyDrive/PDSEI/data/Pesos.txt'
output_path = '/content/drive/MyDrive/PDSEI/resultados2_Media_Moda.txt'

# Factor final
definir_factor_final = 0.22083622891943683

# Llamar a la función
calcular_rmse_y_generar_archivo(definir_factor_final, directorio_imagenes, directorio_txt, pesos_path, output_path)


Línea inválida en archivo de pesos: filename,weight
RMSE total: 323.74 g
Top 5 RMSE más bajos:
B_CUY_01_101_04: 1.83 g
B_CUY_01_014_08: 5.04 g
CUY_01_101_06: 5.68 g
B_CUY_01_014_09: 5.92 g
CUY_01_024_04: 6.24 g

Top 5 RMSE más altos:
B_CUY_01_065_10: 797.13 g
CUY_00_087_06: 856.04 g
CUY_00_087_03: 890.49 g
CUY_00_087_04: 911.69 g
CUY_00_087_02: 953.43 g

Porcentaje de RMSE > 200 g: 53.12%
Porcentaje de RMSE < 50 g: 11.46%
